In [1]:
import Heuristic_model
import pymongo
import config

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
uri = "mongodb://celebal:bZPUhXkQDcdWioAIiwECCdSEiZL3zmQ6bojzYjdiDxQlHhBgzKrJjiuYCWtEbSB4QcinajhByNwKbWlsRoBQ0A==@celebal.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@celebal@"
client = pymongo.MongoClient(uri)
mongo_data1 = client.get_database(name="geography_sizing").get_collection(name='company_data_with_id')
mongo_data1

Collection(Database(MongoClient(host=['celebal.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@celebal@', tls=True), 'geography_sizing'), 'company_data_with_id')

In [5]:
'''
cursor = mongo_data1.find({})
for doc in cursor:
    mongo_data1.delete_one({'_id': doc['_id']})
'''


In [15]:
import pandas as pd
df = pd.read_csv('/home/celebal/Data/genome_prod_20220113-companies.csv')
df.head()

,CompanyId,CompanySlug,CompanyName,Domain,EntityType,IsRelevant,Website,AddressLine1,AddressLine2,AddressCity,AddressState,AddressZipcode,AddressCountry,AddressPhone,ParentCompanyId,StockExchangeId,TaxonomyIds
0,2,3dhistech,3DHISTECH,NaN,COMPANY,1.0,https://www.3dhistech.com,Konkoly-Thege Miklós út 29-33.,18 épület,Budapest,NaN,1121,HU,36 1 392 2274,NaN,0,84
1,3,aacr-publications,AACR Publications,NaN,COMPANY,1.0,https://www.aacr.org,615 Chestnut Street,17th Floor,Philadelphia,PA,19106-4404,US,215-440-9300,NaN,0,105|106
2,4,aacrcentral,AACRcentral,NaN,COMPANY,1.0,https://www.aacr.org,615 Chestnut Street,17th Floor,Philadelphia,PA,19106-4404,US,215-440-9300,NaN,0,105
3,5,abbomax-inc,"AbboMax, Inc.",NaN,COMPANY,1.0,https://www.abbomax.com,"Suite 8, 2528 Qume Drive",NaN,San Jose,CA,95131,US,NaN,NaN,0,87|98
4,7,abs-inc,ABS Inc.,NaN,COMPANY,1.0,https://www.absbio.com,O' Hara House,3 Bermudiana Road,Wilmington,DE,19801,US,NaN,NaN,NaN,87|98


In [19]:
df.iloc[0,[0,2]]

CompanyId              2
CompanyName    3DHISTECH
Name: 0, dtype: object

In [23]:
c = 0
for i in range(len(df)):
    c += 1
    if c == 3:
        break
    id = df.iloc[i,0]
    name = df.iloc[i, 2]
    web = df.iloc[i, 6]
    rec = {'_id' : id, 'company_name' : name, 'Company_url' : web, 'Linkedin': '', 'Glassdoor' : '', 'Website': ''}

    print(rec)
    

{'_id': 2, 'company_name': '3DHISTECH', 'Company_url': 'https://www.3dhistech.com', 'Linkedin': '', 'Glassdoor': '', 'Website': ''}
{'_id': 3, 'company_name': 'AACR Publications', 'Company_url': 'https://www.aacr.org', 'Linkedin': '', 'Glassdoor': '', 'Website': ''}


In [3]:
lc = config.Local_config()
obj1 = Heuristic_model.Model(lc.CENSUS_AGE_PATH,lc.CENSUS_INCOME_PATH,lc.CENSUS_POP_PATH)

In [4]:
import pickle
outputs = []
cursor = mongo_data1.find({})
c=0
assumed_revenue = 100
for document in cursor:
    c += 1
    res = {}
    res['_id'] = document['_id']
    if 'error' in document['Linkedin'].keys():
        res['error'] = 'company not found on linkedin'
    elif document['Linkedin']['sales_data'] == None:
        res['error'] = 'Sales data not available'
    #elif :

    else:
        Country_Dist,State_Dist = obj1.Distribute(document['Linkedin']['sales_data'], document['Linkedin']['locations_data'], document['Glassdoor'],document['Website'],assumed_revenue)
        #res['Country_Dist'] = Country_Dist
        #res['State_Dist'] = State_Dist
        #pickle.dump(res, open('outputs/Result.pkl', 'wb'))
        #print(document['_id'])
        outputs.append({"Country":Country_Dist,"State":State_Dist})
        if c > 60:
            break
pickle.dump(outputs , open("test_model.pkl","wb"))

In [5]:
df=pickle.load(open("test_model.pkl","rb"))

In [8]:
df[10]

{'Country': {'us': 100.0},
 'State': {'us': {'NY': 33.35628485054832,
   'CA': 33.55175091853799,
   'OH': 33.091964230913696}}}

In [10]:
import pickle
pickle.load(open('/home/celebal/Pipeline/outputs/context_output.pkl','rb'))

[{'Error': 'Sales data not available'},
 {'Country': {'gb': 66.66666666666666, 'us': 33.33333333333333}, 'State': {}},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'},
 {'Error': 'company not found on linkedin'}]